In [7]:
def imports():
    global math, np, pd, random, json, torch, Dataset, DataLoader, tqdm, plt, yttm
    
    import math
    import numpy as np
    import pandas as pd

    import random
    import json
    import torch
    from torch.utils.data import Dataset, DataLoader

    from tqdm import tqdm

    from matplotlib import pyplot as plt

    import youtokentome as yttm

In [8]:
imports()

In [9]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
DEVICE

device(type='cuda')

### Подготовка данных

In [2]:
qa_data = list()

with open('qa_data.jsonl') as file_object:
    for line in file_object:
        qa_data.append(json.loads(line.strip()))

NameError: name 'json' is not defined

In [10]:
from collections import deque

questin_answer_data = []
for question_answer in qa_data:
    questin_answer_data.append(question_answer['question'])
    deque(map(questin_answer_data.append, question_answer['responses']))

In [11]:
questin_answer_data[:5]

['долго ли идут деньги с яндексденег на карту visa?',
 'нет. прорыв 35 ;)',
 'можно ли зарегистрировать авто в другом регионе',
 'можно на родственника из того региона.. .  а потом ездить по доверке',
 'что делать если у меня очень тонкие ногти а хочется их отрастить?']

In [12]:
with open('for_bpe.txt', 'w') as f:
    f.write('\n'.join(questin_answer_data))

In [13]:
del questin_answer_data

In [14]:
!head for_bpe.txt

долго ли идут деньги с яндексденег на карту visa?
нет. прорыв 35 ;)
можно ли зарегистрировать авто в другом регионе
можно на родственника из того региона.. .  а потом ездить по доверке
что делать если у меня очень тонкие ногти а хочется их отрастить?
витамины и умная эмаль (каждый день)
ванночки с морской солью. с вечера мажь ногти сверху йодом. не бойся, до утра все впитается.
умная эмаль, витамины, йод, и поменьше крась лаком 
лаки фирмы trind производство usa + кальций
в чем отличие медитации от йоги?


In [29]:
VOCAB_SIZE = 30_000
MODEL_PATH = 'pretrained_bpe_lm.model'

In [16]:
yttm.BPE.train(data='for_bpe.txt', vocab_size=VOCAB_SIZE, model=MODEL_PATH)

In [17]:
tokenizer = yttm.BPE(model=MODEL_PATH)

In [18]:
questions = []
answers = []

for qa in qa_data:
    for answer in qa['responses']:
        questions.append(qa['question'])
        answers.append(answer)

In [19]:
del qa_data

In [23]:
batch_size = 256
tokenized_questions = []

for i_batch in tqdm(range(math.ceil(len(questions) / batch_size))):
    tokenized_questions.extend(
        tokenizer.encode(
            list(questions[i_batch*batch_size:(i_batch+1)*batch_size]),
            bos=True, eos=False,
        )
    )

100%|██████████| 30341/30341 [01:07<00:00, 448.48it/s]


In [24]:
# как сложно без gc
del questions

In [25]:
tokenized_answers = []

for i_batch in tqdm(range(math.ceil(len(answers) / batch_size))):
    tokenized_answers.extend(
        tokenizer.encode(
            list(answers[i_batch*batch_size:(i_batch+1)*batch_size]),
            bos=True, eos=False,
        )
    )

100%|██████████| 30341/30341 [01:03<00:00, 480.40it/s]


In [26]:
del answers

In [29]:
# у меня не хватает памяти, лучше сохраниться
import pickle

with open('questions', 'wb') as f:
    pickle.dump(tokenized_questions, f)

with open('answers', 'wb') as f:
    pickle.dump(tokenized_answers, f)

In [30]:
del tokenized_questions
del tokenized_answers

### Датасет

In [1]:
import pickle

# будем брать одну десятую датасета, иначе памяти не хватит
with open('questions', 'rb') as f:
    questions = pickle.load(f)
questions = questions[:int(len(questions)/10)]

In [2]:
with open('answers', 'rb') as f:
    answers = pickle.load(f)
answers = answers[:int(len(answers)/10)]

In [3]:
print(len(questions), len(answers))
assert len(questions) == len(answers)

776713 776713


In [4]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          14961        7000        4355         697        3606        6941
Swap:         16134        2934       13200


In [11]:
imports()

In [26]:
PAD_INDEX = 0
EOS_INDEX = 3

In [13]:
class SequenceBucketingData(torch.utils.data.Dataset):
    """по сути то же, что в условии, только другие сиквенсы"""
    def __init__(self, questions, answers, max_len, pad_index=PAD_INDEX, eos_index=EOS_INDEX):
        self.questions = questions
        self.answers = answers
        if len(questions) != len(answers):
            raise ValueError('Вопросы и ответы должны быть одной длины')
        self.max_len = max_len
        self.pad_index = pad_index
        self.eos_index = eos_index
        
    def __len__(self):
        return len(self.questions)
    
    def _prepare_sample(self, sequence_q, sequence_a, max_len_q, max_len_a):
        sequence_q = sequence_q[:max_len_q]
        sequence_a = sequence_a[:max_len_a]
        x = sequence_q
        y = sequence_a
        pads_x = [self.pad_index] * (max_len_q - len(x))
        pads_y = [self.pad_index] * (max_len_a - len(y))
        x += pads_x
        y += pads_y
        return x, y
    
    def __getitem__(self, index):
        batch_q = self.questions[index]
        batch_a = self.answers[index]
        max_len_q = min([
            self.max_len,
            max(map(len, batch_q)),
        ])
        max_len_a = min([
            self.max_len,
            max(map(len, batch_a)),
        ])
        batch_x = []
        batch_y = []
        for sample_q, sample_a in zip(batch_q, batch_a):
            x, y = self._prepare_sample(sample_q, sample_a, self.max_len, self.max_len)
            batch_x.append(x)
            batch_y.append(y)
        batch_x = torch.tensor(batch_x).long().to(DEVICE)
        batch_y = torch.tensor(batch_y).long().to(DEVICE)
        return batch_x, batch_y

In [14]:
questions = sorted(questions, key=len)
answers = sorted(answers, key=len)

In [27]:
# сделаем батч побольше
BATCH_SIZE = 256
MAX_LEN = 32

In [16]:
batches_q = []
batches_a = []

for i_batch in range(math.ceil(len(questions) / BATCH_SIZE)):
    q = questions[i_batch*BATCH_SIZE:(i_batch+1)*BATCH_SIZE]
    a = answers[i_batch*BATCH_SIZE:(i_batch+1)*BATCH_SIZE]
    if len(q) != BATCH_SIZE or len(a) != BATCH_SIZE:
        continue
    batches_q.append(questions[i_batch*BATCH_SIZE:(i_batch+1)*BATCH_SIZE])
    batches_a.append(answers[i_batch*BATCH_SIZE:(i_batch+1)*BATCH_SIZE])

In [17]:
# у нас же нет гарантии, что количество вопросов делится на батч-сайз
# мы лишнее убрали условием сверху, надо убедиться
all(
    map(lambda b: True if len(b) == BATCH_SIZE else False, batches_q)
)

True

In [18]:
validation_start_index = int(len(batches_q) * 0.05)

In [19]:
train_seq = SequenceBucketingData(
    questions=batches_q[:-validation_start_index],
    answers=batches_a[:-validation_start_index],
    max_len=MAX_LEN)
test_seq = SequenceBucketingData(
    questions=batches_q[-validation_start_index:],
    answers=batches_a[-validation_start_index:],
    max_len=MAX_LEN)

In [20]:
train_loader = torch.utils.data.DataLoader(train_seq, batch_size=BATCH_SIZE)
validation_loader = torch.utils.data.DataLoader(test_seq, batch_size=BATCH_SIZE)

### Модель

In [21]:
# это специальный дропаут для реккуретных сетей
# хорошо это объясняется здесь: https://youtu.be/WLaAIYQHHMU?t=1093

class SpatialDropout(torch.nn.Dropout2d):
    
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p
    
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T)
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

In [22]:
# сильно переписанный вариант из документации
class Encoder(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = torch.nn.Embedding(input_size, hidden_size)
        self.gru = torch.nn.GRU(hidden_size, hidden_size)

    def forward(self, x):
        embedded = self.embedding(x)
        _, hidden = self.gru(embedded)
        return hidden


In [23]:
# тоже сильно переписанный вариант из документации
class Decoder(torch.nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = torch.nn.Embedding(output_size, hidden_size)
        self.gru = torch.nn.GRU(hidden_size, hidden_size)
        self.out = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = x.unsqueeze(0)
        embedded = self.embedding(x)
        outputs, hidden = self.gru(embedded, hidden)
        output = self.softmax(self.out(outputs[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=DEVICE)

In [24]:
class Seq2Seq(torch.nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, source, target):
        target_len = target.shape[0]
        outputs = torch.zeros(target_len, BATCH_SIZE, VOCAB_SIZE).to(DEVICE)
        hidden = self.encoder(source)
        
        x = target[0]
        for t in range(1, target_len):
            output, hidden = self.decoder(x, hidden)
            outputs[t] = output
            x = output.argmax(1)
        return outputs
        

In [30]:
encoder = Encoder(VOCAB_SIZE, 300)
decoder = Decoder(300, VOCAB_SIZE)
seq2seq = Seq2Seq(encoder, decoder)

In [31]:
def train(
    train_loader, test_loader,
    seq2seq,
    encoder_criterion=torch.nn.CrossEntropyLoss(ignore_index=PAD_INDEX),
    decoder_criterion=torch.nn.CrossEntropyLoss(ignore_index=PAD_INDEX),
    encoder_optimizer=torch.optim.Adam(params=encoder.parameters()),
    decoder_optimizer=torch.optim.Adam(params=decoder.parameters()),
):
    for e in range(10):
        print(f'Epoch {e}')
        seq2seq.train()
        for x, y in train_loader:
            output = seq2seq.forward(x, y)
    

In [ ]:
train(train_loader, None, seq2seq)

Epoch 0


In [44]:
encoder_criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_INDEX)
decoder_criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_INDEX)
encoder_optimizer = torch.optim.Adam(params=encoder.parameters())
decoder_optimizer = torch.optim.Adam(params=decoder.parameters())

In [ ]:
def run_training(train_loader, test_loader, epochs=10, last_n_losses=500):
    losses = []
    best_validation_loss = 1e+6
    
    for e in range(epochs):
        pass